In [2]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


#  Modelado y evaluación
# -----------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRFRegressor
import xgboost as xgb

In [3]:
df = pd.read_pickle("../data/bikes_casuales_estand_encod.pkl")
df.head()

,año,mes,festivo,dia,meteo,temperatura,humedad,velo_viento,usu_casuales
0,0,0,1,3,1,-0.486274,0.858854,-0.208909,331
1,0,0,0,2,1,-0.425690,0.331548,0.687550,131
2,0,0,0,1,2,-0.949970,-0.912000,0.685210,120
3,0,0,0,1,2,-0.938563,-0.176088,-0.210435,108
4,0,0,0,0,2,-0.853992,-0.913519,0.060294,82


In [4]:
X = df.drop("usu_casuales", axis = 1)
y = df["usu_casuales"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 15)

In [5]:
# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

/home/arillien/anaconda3/envs/Proyecto3/lib/python3.10/site-packages/xgboost/data.py:463: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  or is_sparse(dtype)
/home/arillien/anaconda3/envs/Proyecto3/lib/python3.10/site-packages/xgboost/data.py:464: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  or (is_categorical_dtype(dtype) and enable_categorical)
/home/arillien/anaconda3/envs/Proyecto3/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/arillien/anaconda3/envs/Proyecto3/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) inste

In [6]:
m = len(X.columns)

params = dict(
    objective="reg:squarederror",
    learning_rate=1,
    num_parallel_tree=500,
    subsample=0.63,
    colsample_bynode=int(np.sqrt(m))/m,
    reg_lambda=0,
    max_depth=20,
    min_child_weight=2
)

In [7]:
rf_xgb = xgb.train(  # Wall time 34 s
    params, 
    dtrain_reg, 
    num_boost_round=1
)

In [8]:
y_pred_test_xgbrf = rf_xgb.predict(dtest_reg)
y_pred_train_xgbrf = rf_xgb.predict(dtrain_reg)

In [9]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [10]:
xgbrf_results = metricas(y_test, y_train, y_pred_test_xgbrf, y_pred_train_xgbrf, "XGBoost RF")
xgbrf_results

,MAE,MSE,RMSE,R2,set,modelo
0,280.910536,178037.978532,421.945469,0.671771,test,XGBoost RF
1,176.080128,69610.912914,263.838801,0.846167,train,XGBoost RF
